## 1. Importing Data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Define the path to import data
path = r"C:\Users\enthu\Desktop\Bootcamps\Instacart Basket Analysis"

In [3]:
path

'C:\\Users\\enthu\\Desktop\\Bootcamps\\Instacart Basket Analysis'

In [4]:
# Import ords_prods_merge
ords_prods_merge = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_products_merged_derived.pkl'))

## 2. Grouping and Aggregating Data

In [5]:
# Get the means of the "order_number" column and split them into groups based on "department_id"
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457687
2,17.277920
3,17.179756
4,17.811403
5,15.213779
6,16.439806
7,17.225773
8,15.340520


Since the subset was only the first one million rows, it was missing a lot of depatment_id. The entire dataframe also has different means because it includes much more rows.

In [6]:
# Create a column named max_order used to store the maximum order_number grouped bny user_id
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\enthu\AppData\Local\Temp\ipykernel_19056\1004745265.py:2: FutureWarning: The provided callable <function max at 0x00000219F4D344A0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [7]:
ords_prods_merge.head(15)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,prior,1,2,8,NaN,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,prior,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,10
2,473747,1,prior,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Most orders,10
3,2254736,1,prior,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders,10
4,431534,1,prior,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Most orders,10
5,3367565,1,prior,6,2,7,19.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10
6,550135,1,prior,7,1,9,20.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10
7,3108588,1,prior,8,1,14,14.0,196,2,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10
8,2295261,1,prior,9,1,16,0.0,196,4,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10
9,2550362,1,prior,10,4,8,30.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders,10


In [8]:
# Create a new column named loyalty_flag and assign different flags to it
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15891077
Loyal customer      10293737
New customer         6249398
Name: count, dtype: int64

In [10]:
# Get the means, mins, and maxes of the order_number column and split them into groups based on "loyalty_flag"
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.388747  1.0  99999.0
New customer      13.294370  1.0  99999.0
Regular customer  12.496203  1.0  99999.0

The prices of products purchased by different customers differ from each other.

In [11]:
# Create a column named average_price used to store the averages of prices grouped by user_id
ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\enthu\AppData\Local\Temp\ipykernel_19056\3131404533.py:2: FutureWarning: The provided callable <function mean at 0x00000219F4D34EA0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [12]:
ords_prods_merge.head(15)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797
2,473747,1,prior,3,3,12,21.0,196,1,1,...,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797
4,431534,1,prior,5,4,15,28.0,196,1,1,...,77,7,9.0,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797
6,550135,1,prior,7,1,9,20.0,196,1,1,...,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,77,7,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797


In [13]:
# Create a new column named spenders and assign different flags to it
ords_prods_merge.loc[ords_prods_merge['average_price'] < 10, 'spenders'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['average_price'] >= 10, 'spenders'] = 'High spender'

In [14]:
# Check the amount of different spenders
ords_prods_merge['spenders'].value_counts(dropna = False)

spenders
Low spender     31798751
High spender      635461
Name: count, dtype: int64

In [15]:
# Create a column named frequency_median used to store the medians of frequency grouped bny user_id
ords_prods_merge['frequency_median'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\enthu\AppData\Local\Temp\ipykernel_19056\3634700989.py:2: FutureWarning: The provided callable <function median at 0x00000219F4E7CCC0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['frequency_median'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [16]:
ords_prods_merge.head(15)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spenders,frequency_median
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5
2,473747,1,prior,3,3,12,21.0,196,1,1,...,9.0,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,9.0,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5
4,431534,1,prior,5,4,15,28.0,196,1,1,...,9.0,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5
6,550135,1,prior,7,1,9,20.0,196,1,1,...,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,9.0,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,20.5
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,9.0,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5


In [17]:
# Create a new column named customers and assign different flags to it
ords_prods_merge.loc[ords_prods_merge['frequency_median'] > 20, 'customers'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['frequency_median'] > 10) & (ords_prods_merge['frequency_median'] <= 20), 'customers'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['frequency_median'] <= 10, 'customers'] = 'Frequent customer'

In [18]:
# Check the amount of different types of customers
ords_prods_merge['customers'].value_counts(dropna = False)

customers
Frequent customer        21577409
Regular customer          7217134
Non-frequent customer     3639669
Name: count, dtype: int64

In [19]:
ords_prods_merge.to_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_products_grouped_aggregated.pkl'))